In [11]:
import requests
import torch
from transformers import BlipForConditionalGeneration, BlipProcessor
from diffusers import DDIMScheduler, DDIMInverseScheduler, StableDiffusionPix2PixZeroPipeline
from PIL import Image
import numpy as np
import pickle
import glob
from pathlib import Path
import os

def download(embedding_url, local_filepath):
    r = requests.get(embedding_url)
    with open(local_filepath, "wb") as f:
        f.write(r.content)

In [2]:
captioner_id = "Salesforce/blip-image-captioning-base"
processor = BlipProcessor.from_pretrained(captioner_id)
model = BlipForConditionalGeneration.from_pretrained(captioner_id, torch_dtype=torch.float16, low_cpu_mem_usage=True)


In [3]:
sd_model_ckpt = "CompVis/stable-diffusion-v1-4"
pipeline = StableDiffusionPix2PixZeroPipeline.from_pretrained(
    sd_model_ckpt,
    caption_generator=model,
    caption_processor=processor,
    torch_dtype=torch.float16,
    safety_checker=None,
)
pipeline.scheduler = DDIMScheduler.from_config(pipeline.scheduler.config)
pipeline.inverse_scheduler = DDIMInverseScheduler.from_config(pipeline.scheduler.config)
pipeline.enable_model_cpu_offload()

Loading pipeline components...: 100%|█████████████████| 6/6 [00:00<00:00,  7.27it/s]
You have disabled the safety checker for <class 'diffusers.pipelines.deprecated.stable_diffusion_variants.pipeline_stable_diffusion_pix2pix_zero.StableDiffusionPix2PixZeroPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .
The config attributes {'skip_prk_steps': True} were passed to DDIMInverseScheduler, but are not expected and will be ignored. Please verify your scheduler_config.json configuration file.


In [9]:
def get_caption(image):
    caption = pipeline.generate_caption(image)
    return caption

In [4]:
def get_reconstructed_image(raw_image):
    caption = get_caption(raw_image)
    generator = torch.manual_seed(0)
    inv_latents = pipeline.invert(caption, image=raw_image, generator=generator).latents
    source_embeds = pipeline.get_embeds(caption, batch_size=2)
    pred_img = pipeline(
            caption,
            source_embeds=source_embeds,
            target_embeds=source_embeds,
            num_inference_steps=50,
            cross_attention_guidance_amount=0.15,
            generator=generator,
            latents=inv_latents,
            negative_prompt=caption,
        ).images[0]
    
    return pred_img, caption, inv_latents

In [5]:
def calculate_psnr(original_image, reconstructed_image):
    # Convert images to numpy arrays
    original_array = np.array(original_image)
    reconstructed_array = np.array(reconstructed_image)

    # Calculate mean squared error
    mse = np.mean((original_array - reconstructed_array) ** 2)

    # Maximum possible pixel value
    max_pixel_value = 255  # Assuming 8-bit images

    # Calculate PSNR
    psnr = 20 * np.log10(max_pixel_value / np.sqrt(mse))

    return psnr

In [12]:
def calculate_bpp(data_path, image_size):
    # Get the file size of the image in bytes
    file_size_bytes = os.path.getsize(data_path)

    # Convert file size to bits
    file_size_bits = file_size_bytes * 8

    width, height = image_size

    # Calculate total number of pixels
    total_pixels = width * height

    # Calculate bits per pixel
    bpp = file_size_bits / total_pixels

    return bpp

In [13]:
img_files =sorted(glob.glob('Jockey/*.png'))
psnr_values =[]
bpp_values = []
for file in img_files:
    frame = Image.open(file).convert("RGB").resize((512, 512))
    pred , cap, latent = get_reconstructed_image(frame)
    psnr = calculate_psnr(frame,pred)   
    psnr_values.append(psnr)
    pred.save('compressed/'+ Path(file).stem+ '.png')
    output_name = 'compressed/'+ Path(file).stem + '.pkl'
    with open(output_name, 'wb') as f:
        pickle.dump({'caption': cap, 'tensor': latent}, f)
    bpp = calculate_bpp(output_name,(512,512))
    bpp_values.append(bpp)
    print('processing complete:' ,file)


100%|███████████████████████████████████████████████| 50/50 [00:13<00:00,  3.77it/s]


processing complete: Jockey/img00001.png


100%|███████████████████████████████████████████████| 50/50 [00:13<00:00,  3.73it/s]


processing complete: Jockey/img00002.png


100%|███████████████████████████████████████████████| 50/50 [00:13<00:00,  3.74it/s]


...

processing complete: Jockey/img00099.png


100%|███████████████████████████████████████████████| 50/50 [00:13<00:00,  3.72it/s]


processing complete: Jockey/img00100.png


 26%|████████████▏                                  | 13/50 [00:07<00:20,  1.82it/s]


KeyboardInterrupt: 

In [16]:
bpp = sum(bpp_values)/len(bpp_values)

1.0147705078125

In [17]:
psnr = sum(psnr_values)/len(psnr_values)

In [18]:
psnr

31.497359305606697